### Optimal Portfolio Allocation



An investment universe of the following risky assets with a dependence structure (correlation) is given:

\begin{equation*}

\begin{matrix} 
\textbf{Asset} & \boldsymbol\mu & \boldsymbol\sigma & \boldsymbol\omega\\
A & 0.02 & 0.05 & \omega_1\\
B & 0.07 & 0.12 & \omega_2\\
C & 0.15 & 0.17 & \omega_3\\
D & 0.20 & 0.25 & \omega_4
\end{matrix}
\qquad
\qquad
R =
\begin{pmatrix}
1 & 0.3 & 0.3 & 0.3\\
0.3 & 1 & 0.6 & 0.6\\
0.3 & 0.6 & 1 & 0.6\\
0.3 & 0.6 & 0.6 & 1
\end{pmatrix}

\end{equation*}

**Question 1.** Consider minimum variance portfolio with a target return m. 

\begin{equation*}
\underset{\boldsymbol\omega}{\arg\min} \frac{1}{2}\omega'\Sigma\omega
\qquad
s.t.
\quad
\omega'1 = 1
\quad
\mu_\Pi = \omega'\mu = m
\end{equation*}


- Formulate the Lagrangian and give its partial derivatives. 
- Write down the analytical solution for optimal allocations $ \omega^* $ (derivation not required).
- Inverse optimisation: generate above 700 random allocation sets (vectors) 4 × 1, those will not be optimal allocations. 
    - Standardise each set to satisfy $\omega'1 = 1$, in fact you can generate 3 allocations and compute the 4th. 
    - For each vector of allocations compute $\mu_\Pi = \omega'\mu$ and $\sigma_\Pi = \sqrt{\omega'\Sigma\omega}$. 
    - Plot the cloud of points of $\mu_\Pi$ vertically on $\sigma_\Pi$ horizontally. <ins>Explain this plot</ins>. 
    
Hint: Since we treat this as an inverse optimisation, there is no computation of $\omega^*$ from the ready formula.

### Answers and code

Form the Langrange function with two multipliers $\lambda$ and $\gamma$ for a given vector of weights $w$
\begin{equation*}
L(w, \lambda, \gamma) = \frac{1}{2}w'\Sigma w + \lambda(m - w'\mu) + \gamma(1 - w'1)
\end{equation*}

where $(m - w'\mu)$ is the return constraint and $(1 - w'1)$ is the budget constraint.


##### First order condition

The optimal portfolio is the point where the gradient of $L(w, \lambda, \gamma) = 0$.

Therefore $\frac{\partial L}{\partial w} = 0$.

\begin{equation*}
\frac{\partial L}{\partial w} = \frac{1}{2} 2 \Sigma w + \lambda (-\mu) + \gamma(-1) = \Sigma w - \lambda \mu - \gamma 1 = 0
\end{equation*}

##### Second order condition

Given this is a minimisation problem, we seek the Hessian to be greater than 1

\begin{equation*}
\frac{\partial^2 L}{\partial w^2} = \Sigma
\end{equation*}

As teh covariance matrix is positive, the 2nd order condition is satisfied. 

Therefore the optimal allocation $w^*$ can be derived 

\begin{equation*}
\Sigma w - \lambda \mu - \gamma 1 = 0 \newline
\end{equation*}

\begin{equation*}
w^* = \Sigma^{-1} (\lambda \mu + \gamma 1)
\end{equation*}

Substituting the above optimal weight $w^*$ into the constraint 

\begin{equation*}
\mu'w = \mu' \Sigma^{-1} (\lambda \mu + \gamma 1) = \mu' \Sigma^{-1} \lambda \mu + \mu' \Sigma^{-1}\gamma 1 = m
\end{equation*}
and 
\begin{equation*}
1'w = 1' \Sigma^{-1} (\lambda \mu + \gamma 1) = 1' \Sigma^{-1} \lambda \mu + 1' \Sigma^{-1}\gamma 1 = 1
\end{equation*}

Setting scalars for ease of presentation and computation
\begin{equation*}
A = 1\Sigma^{-1} 1'
\end{equation*}

\begin{equation*}
B = \mu' \Sigma^{-1} 1 = 1' \Sigma^{-1} \mu
\end{equation*}

\begin{equation*}
C = \mu' \Sigma^{-1} \mu
\end{equation*}

The constraint functions then become

\begin{equation*}
C \lambda + B \gamma = m
\end{equation*}

\begin{equation*}
B \lambda + A \gamma = 1
\end{equation*}

This is a system of 2 equations with 2 unknowns

\begin{equation*}
\lambda = \frac{m - B \gamma}{C}
\end{equation*}

therefore

\begin{equation*}
\frac{Bm - B^2 \gamma}{C} + A \gamma = 1
\end{equation*}

\begin{equation*}
Bm - B^2 \gamma + A C \gamma = C
\end{equation*}

\begin{equation*}
A C \gamma - B^2 \gamma = C - Bm
\end{equation*}

\begin{equation*}
\gamma = \frac{C - Bm}{AC - B^2}
\end{equation*}

in the same manner

\begin{equation*}
\lambda = \frac{Am - B}{AC - B^2}
\end{equation*}


Finally, substituting back into optimal weight $w^*$

\begin{equation*}
w^* = \Sigma^{-1} (\lambda \mu + \gamma 1) = \frac{1}{AC - B^2} \Sigma^{-1}[(A \mu - B1)m + (C1 - B \mu)]
\end{equation*}


In [ ]:
# Import the relevant packages

In [ ]:
# Set values for N and initiate an empty list for the 
# Generate N random weights
N = 700

# Standardise these weights

# Add them to a dataframe